# Basal Ganglia on SpiNNaker and SpineML

Comparing event based and impulse based cut-down models with predictable connectivity and Poisson inputs.

## Simple analysis from rate

This is very simple spike counting code, useful for constant input, steady state network analysis.

In [4]:
import numpy as np
def firing_rate (logdir1, spikelogfile, numneurons, simlength_ms, offset_ms=0):
    # Read first lot of data
    logpath = logdir1 + spikelogfile
    spikelist = np.genfromtxt (logpath, delimiter=',')
    
    if spikelist.size == 0:
        print ('Total rate in {0} neurons in {1}: 0 (empty log file)'.format(numneurons, spikelogfile))
        return 0, 0
    #print ('spikelist.shape: {0}'.format(spikelist.shape))
    # Number of rate total / number of neurons / time
    if spikelist.shape[0] == 2:
        trate = 1
    else:
        trate = len(spikelist[:,0])
        #print ('trate is {0}'.format(trate))
        sli = np.where(spikelist[:,0]>offset_ms)
        #print (spikelist[sli,0])
        trate = len(spikelist[sli,0].T)
        #print ('trate is {0}'.format(trate))

    print ('Total rate in {0} neurons in {1}: {2}'.format(numneurons, spikelogfile, trate))
    
    spnps = float(trate) / float(numneurons) / float(float(simlength_ms)/1000.0)
    print ('Mean spike rate: {0}'.format(spnps))
    
    return trate, spnps
    
    # I'd like to implement a moving bin to get a better estimate of the stats on the spike rate.
    # Shouldn't be too hard, except that as the spike rate is so low in some populations, I'd probably need
    # to run the model for much longer, so perhaps there's no great advantage over just running the model ten times.
    

In [10]:
# Event Based Model
firing_rate('/home/seb/src/SpineML_2_BRAHMS/temp/EventBased_e0/log/','D1_MSN_spike_log.csv', 1255, 6000)
firing_rate('/home/seb/src/SpineML_2_BRAHMS/temp/EventBased_e0/log/','FSI_spike_log.csv', 84, 6000)

Total rate in 1255 neurons in D1_MSN_spike_log.csv: 2961
Mean spike rate: 0.393227091633
Total rate in 84 neurons in FSI_spike_log.csv: 684
Mean spike rate: 1.35714285714


(684, 1.357142857142857)

In [11]:
# Impulse Based Model
firing_rate('/home/seb/src/SpineML_2_BRAHMS/temp/ImpulseBased_e0/log/','D1_MSN_spike_log.csv', 1255, 6000)
firing_rate('/home/seb/src/SpineML_2_BRAHMS/temp/ImpulseBased_e0/log/','FSI_spike_log.csv', 84, 6000)

Total rate in 1255 neurons in D1_MSN_spike_log.csv: 2832
Mean spike rate: 0.37609561753
Total rate in 84 neurons in FSI_spike_log.csv: 691
Mean spike rate: 1.37103174603


(691, 1.371031746031746)

## Batch running

In [25]:
def runsim (spineml2brahmsdir, modelbasedir, modeldir, expt=0):

    modelpath = modelbasedir + modeldir
    spineml2brahmsdir ='/home/seb/src/SpineML_2_BRAHMS'
    spinemltmp = spineml2brahmsdir+'/temp/'+ modeldir
    
    # Execute the model
    import os
    spinemlcmd = '/bin/bash -c "pushd '+spineml2brahmsdir+' && ' +\
                 'LD_LIBRARY_PATH=\"\" ./convert_script_s2b -m ' + modelpath + ' -e' +\
                 `expt` + ' -g -o' + spinemltmp + '; popd"'

    #print spinemlcmd
    os.system(spinemlcmd)


In [27]:
modelbasedir = '/home/seb/models/GPR-BSB/impulse_event_compare/' # NB trailing slash

# Either:
#modeldir = 'event1'
#expt = 0
# Or:
modeldir = 'impulse1'
expt = 0

spineml2brahmsdir ='/home/seb/src/SpineML_2_BRAHMS'
spinemltmp = spineml2brahmsdir+'/temp/'+ modeldir

d1spikes = np.array([])
fsispikes = np.array([])
d1rate = np.array([])
fsirate = np.array([])

for c in range(0,15):

    runsim(spineml2brahmsdir, modelbasedir, modeldir, expt)

    spikes, rate = firing_rate(spinemltmp+'/log/','D1_MSN_spike_log.csv', 1255, 6000)
    d1spikes = np.append(d1spikes, spikes)
    d1rate = np.append(d1rate, rate)
    
    spikes, rate = firing_rate(spinemltmp+'/log/','FSI_spike_log.csv', 84, 6000)
    fsispikes = np.append(fsispikes, spikes)
    fsirate = np.append(fsirate, rate)

# Save the spikes and rates
np.savetxt('/home/seb/models/GPR-BSB/labbook/'+'d1spikes1.csv', d1spikes, fmt='%d', delimiter=',', newline='\n')
np.savetxt('/home/seb/models/GPR-BSB/labbook/'+'fsispikes1.csv', fsispikes, fmt='%d', delimiter=',', newline='\n')

np.savetxt('/home/seb/models/GPR-BSB/labbook/'+'d1rates1.csv', d1rate, fmt='%.8f', delimiter=',', newline='\n')
np.savetxt('/home/seb/models/GPR-BSB/labbook/'+'fsirates1.csv', fsirate, fmt='%.8f', delimiter=',', newline='\n')


Total rate in 1255 neurons in D1_MSN_spike_log.csv: 2509
Mean spike rate: 0.333200531208
Total rate in 84 neurons in FSI_spike_log.csv: 637
Mean spike rate: 1.26388888889
Total rate in 1255 neurons in D1_MSN_spike_log.csv: 3244
Mean spike rate: 0.430810092961
Total rate in 84 neurons in FSI_spike_log.csv: 740
Mean spike rate: 1.46825396825
Total rate in 1255 neurons in D1_MSN_spike_log.csv: 2888
Mean spike rate: 0.383532536521
Total rate in 84 neurons in FSI_spike_log.csv: 650
Mean spike rate: 1.28968253968
Total rate in 1255 neurons in D1_MSN_spike_log.csv: 2978
Mean spike rate: 0.395484727756
Total rate in 84 neurons in FSI_spike_log.csv: 660
Mean spike rate: 1.30952380952
Total rate in 1255 neurons in D1_MSN_spike_log.csv: 2772
Mean spike rate: 0.36812749004
Total rate in 84 neurons in FSI_spike_log.csv: 648
Mean spike rate: 1.28571428571
Total rate in 1255 neurons in D1_MSN_spike_log.csv: 2759
Mean spike rate: 0.366401062417
Total rate in 84 neurons in FSI_spike_log.csv: 671
Mean s